In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
import pathlib
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import pymysql

from sklearn.model_selection import train_test_split
import pickle

In [2]:
data_path = "..\\student_images\\"

In [3]:
path = pathlib.Path(data_path)
path

WindowsPath('../student_images')

In [4]:
list(path.glob('*/*.jpg'))[0]

WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143108.jpg')

In [5]:
image_count = len(list(path.glob('*/*.jpg')))
print(image_count)

38


In [6]:
img = list(path.glob('Ananya Rathour/*'))
img[:5]

[WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143108.jpg'),
 WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143111.jpg'),
 WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143114.jpg'),
 WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143119.jpg'),
 WindowsPath('../student_images/Ananya Rathour/IMG_20240317_143122.jpg')]

In [7]:
#PIL.Image.open(str(img[1]))

In [8]:
student_images_dict = {
    'Ananya Rathour': list(path.glob('Ananya Rathour/*')),
    'Manshi Rathour': list(path.glob('Manshi Rathour/*')),
    'Naincy Rathour': list(path.glob('Naincy Rathour/*')),
    'Mayank Rathour': list(path.glob('Mayank Rathour/*'))    
}

In [9]:
img = cv2.imread(str(student_images_dict['Ananya Rathour'][0]))

In [10]:
img.shape

(1909, 1432, 3)

## Saving image of new student in new folder and also appending its path in dictionary

In [11]:
def save_images_for_new_student(new_student_name, image_data, data_path):
    # Create a folder for the new student if it doesn't exist
    student_folder_path = os.path.join(data_path, new_student_name)
    os.makedirs(student_folder_path, exist_ok=True)
    
    # Save the images sent from the backend
    image_paths = []
    for i, image in enumerate(image_data):
        image_path = os.path.join(student_folder_path, f"image_{i+1}.jpg")
        with open(image_path, "wb") as file:
            file.write(image)
        image_paths.append(pathlib.Path(image_path))
    
    # Update student_images_dict with the new student folder path
    student_images_dict[new_student_name] = image_paths
    
    return student_images_dict

In [12]:
# new_student_name = "New Student"
# image_data = [b'image_1_data', b'image_2_data', b'image_3_data', b'image_4_data', b'image_5_data', b'image_6_data']
# save_images_for_new_student(new_student_name, image_data, data_path)
# student_images_dict 

## Connecting to MySQL server to fetch student name with their details

In [13]:
def load_data_from_mysql(host, user, password, database, student_images_dict):
    try:
        # Connect to MySQL without specifying the authentication plugin
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Create a cursor object
        cursor = connection.cursor()

        # Execute a query to fetch data
        query = "SELECT * FROM students"
        cursor.execute(query)

        # Fetch data and convert it into a DataFrame
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=[col[0] for col in cursor.description])

        # Convert DataFrame to dictionary
        student_dict = df.set_index('name')['student_id'].to_dict()

        # Close the cursor and connection
        cursor.close()
        connection.close()

        return student_dict

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

    
    
host = 'localhost'
user = 'root'
password = 'root'
database = 'smart_attendance'

student_dict = load_data_from_mysql(host, user, password, database, student_images_dict)


print("Student Dictionary:")
print(student_dict)


Student Dictionary:
{'Manshi Rathour': 1, 'Ananya Rathour': 2, 'Naincy Rathour': 3, 'Mayank Rathour': 4}


## Resizing images 

In [14]:
def resize_images(student_images_dict, student_dict, X=None, y=None, image_size=(180, 180)):
    if X is None:
        X = []
    if y is None:
        y = []

    processed_files = set()
    
    # Add existing files to processed_files
    for img in X:
        processed_files.add(img)
    
    for student_name, images in student_images_dict.items():
        for image in images:
            # Check if the image file exists
            if os.path.exists(str(image)):
                # Check if the image has been processed already
                if str(image) not in processed_files:
                    # Read the image
                    img = cv2.imread(str(image))
                    
                    # Check if the image was read successfully
                    if img is not None:
                        # Resize the image
                        resized_img = cv2.resize(img, image_size)
                        
                        # Convert BGR to RGB
                        resized_img_rgb = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
                        
                        # Append the resized image to X
                        X.append(resized_img_rgb)
                        
                        # Append the corresponding label to y
                        y.append(student_dict[student_name])
                        
                        # Add the filename to processed_files
                        processed_files.add(str(image))
                    else:
                        print(f"Unable to read image: {image}")
                else:
                    print(f"Image already processed: {image}")
            else:
                print(f"Image file does not exist: {image}")

    # Convert lists to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    return X, y


X, y = resize_images(student_images_dict, student_dict)


print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (38, 180, 180, 3)
y shape: (38,)


## Train Test Split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## Processing : scale images 

In [16]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

# Build convolutional neural network and train it

In [17]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(180, 
                                                              180,
                                                              3)),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [18]:
num_classes = 5

model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=30)       



Epoch 1/30


1/1 [==============================] - 2s 2s/step - loss: 1.4892 - accuracy: 0.2500
Epoch 2/30
1/1 [==============================] - 0s 176ms/step - loss: 4.3532 - accuracy: 0.2857
Epoch 3/30
1/1 [==============================] - 0s 175ms/step - loss: 7.2397 - accuracy: 0.2857
Epoch 4/30
1/1 [==============================] - 0s 184ms/step - loss: 4.9593 - accuracy: 0.2500
Epoch 5/30
1/1 [==============================] - 0s 173ms/step - loss: 2.8362 - accuracy: 0.2500
Epoch 6/30
1/1 [==============================] - 0s 180ms/step - loss: 1.5119 - accuracy: 0.5357
Epoch 7/30
1/1 [==============================] - 0s 173ms/step - loss: 1.1736 - accuracy: 0.4643
Epoch 8/30
1/1 [==============================] - 0s 174ms/step - loss: 1.0441 - accuracy: 0.7500
Epoch 9/30
1/1 [==============================] - 0s 171ms/step - loss: 0.9277 - accuracy: 0.5714
Epoch 10/30
1/1 [==============================] - 0s 172ms/step - loss: 0.8829 - accuracy: 0.5714
Epoch 11/30
1/1 [=

In [19]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 227ms/step - loss: 0.4399 - accuracy: 0.9000


[0.4398633539676666, 0.8999999761581421]

In [20]:
predictions = model.predict(X_test_scaled)
predictions

1/1 [==============================] - 0s 134ms/step


array([[-10.613449  ,   2.1189766 ,  -0.53918976,   5.1935725 ,
          1.2792971 ],
       [-11.619715  ,   0.27666864,   5.519066  ,   1.5247225 ,
          3.8101747 ],
       [-13.396381  ,   8.917156  ,  -4.603524  ,   2.1131668 ,
          2.4409087 ],
       [-12.786484  ,   8.450519  ,  -3.9634898 ,   1.8976936 ,
          2.304776  ],
       [-12.2775955 ,   7.233598  ,  -3.2393267 ,   1.5744544 ,
          2.7039654 ],
       [-13.351382  ,   9.072342  ,  -4.213699  ,   2.3564663 ,
          1.9061737 ],
       [-10.416382  ,   2.2982593 ,  -0.89488137,   5.1814146 ,
          1.369103  ],
       [-12.613455  ,   7.1322713 ,  -3.2532377 ,   2.1346457 ,
          2.604177  ],
       [-10.27751   ,   1.9655902 ,   0.11039606,   5.6852584 ,
          0.62262446],
       [-10.940866  ,  -7.660872  ,  23.431868  ,   4.7805014 ,
         -3.0649114 ]], dtype=float32)

In [21]:
score = tf.nn.softmax(predictions[0])

In [22]:
np.argmax(score)

3

In [23]:
y_test[0]

3

## Save trained model to pickel file

In [24]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)